# Benchmark Postgres with Q3C extension

In [ ]:
# First make sure our services are running
! docker compose up -d

In [ ]:
# Now drop the database if it exists and create a table for our
# alerts
sql = """
DROP DATABASE IF EXISTS boom;
CREATE DATABASE boom;
USE boom;
CREATE TABLE IF NOT EXISTS alerts (
    id SERIAL PRIMARY KEY,
    object_id VARCHAR(255) NOT NULL,
    cand_id VARCHAR(255) NOT NULL,
    candidate JSONB NOT NULL,
    ra DOUBLE PRECISION NOT NULL,
    dec DOUBLE PRECISION NOT NULL,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);

-- Create a Q3C index on the ra and dec columns for faster querying
CREATE INDEX idx_objects_q3c ON alerts (q3c_ang2ipix(ra, dec));
"""

In [ ]:
# Insert all the ZTF alerts